In [2]:
import pandas as pd
import sys
sys.path.append("../src")
from preprocessing import clean_text
from classifier import GPTJobDescriptionCategorizer
import os

In [2]:
df = pd.read_csv("../Engineer_20230826.csv")
df['JobDescription'] = df['JobDescription'].apply(clean_text)

In [ ]:
classifier = GPTJobDescriptionCategorizer(job_postings_data=df, n_samples=500)
labelled_training_data = classifier.process_random_postings()

Processing posting 5201/500
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Processing posting 3001/500
Processing posting 8301/500
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Processing posting 2601/500
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.
Error calling GPT: Connection error.


In [3]:
output_directory = '../data'  # Replace with your desired directory
output_filename = 'gpt_training_data.csv'
full_path = os.path.join(output_directory, output_filename)
labelled_training_data.to_csv(full_path, index=False) 

NameError: name 'labelled_training_data' is not defined